# Building a Spelling Corrector

Automatic spelling correction is something we encounter every day, but how it works isn't immediately obvious. With this notebook I aim to show how to build an understandable spelling corrector that, as a bonus, also works reasonably well.

## First, some probability theory
### Equation to maximize
Given a misspelled word `w`, we want to create a function `correct(w)` which returns the word the user was most likely trying to type. As there is unavoidable ambiguity in correcting a word ("ther" could be "their", "there", or "the", just to name a few) we will assign probabilities to each possible correction (candidate). Each candidate `c`'s probability represents the likelihood that given misspelled word `w`, `c` was the intended word: `P(c|w)`. However, this is a difficult probability to evaluate as there are too many factors invovled. Ideally, we want to break down the problem into smaller equations and then combine them.

[Bayes' Theorem](https://en.wikipedia.org/wiki/Bayes'_theorem) tells us that `P(c|w)` is equivalent to `(P(c) * P(w|c)) / (P(w)`. `P(w)` is simply the probability of the misspelled word appearing, which is the same for all candidates so we can ignore it as it doesn't factor into the maximization. This leaves us with the equation: `P(c) * P(w|c)` to maximize over all candidates `c` for a misspelled word `w`.

That was the most complicated bit of this entire exercise, it's all downhill from here!

### Breaking down the equationa
`P(c)`: The probability of the candidate word occurring in an English text. Words like "the", "and" etc. occur far more frequently than words like "monopoly" or "horticulture". This gives them a higher probability of occurring.

`P(w|c)`: The probability that the misspelled word `w` was typed when the author meant `c`. For example, `P("clasroom"|"classroom")` is very high, while `P("cladfalsasrosfsafam"|"classroom")` is very low.

Over all candidates `c`: We can't evaluate the equation for every word in the English language, so instead we'll evaluate it on words that are a couple simple edits away from the misspelled word `w`.

## Candidate selection
We define a simple edit to a word as any of the following:
- Deletion: Remove one letter
- Swap: Swap two adjacent letters
- Substitution: Change one letter to another
- Insertion: Add a new letter

This is implemented by the function `one_edit_from`

In [24]:
LETTERS = "abcdefghijklmnopqrstuvwxyz"

In [25]:
def one_edit_from(word):
    # Generates splits of a word ex. "dog" -> ("", "dog"), ("d", "og"), ("do", "g"), ("dog", "")
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    deletes = [l + r[1:] for (l, r) in splits if r]
    swaps = [l + r[1] + r[0] + r[2:] for (l, r) in splits if len(r) > 1]
    substitutes = [l + x + r[1:] for (l, r) in splits for x in LETTERS]
    inserts = [l + x + r for (l, r) in splits for x in LETTERS]

    return set(deletes + swaps + substitutes + inserts)

In [26]:
len(one_edit_from("dog"))

182

In [27]:
list(one_edit_from("dog"))[:8]

['doh', 'dxog', 'dqog', 'dkg', 'dogj', 'dogg', 'doq', 'dpog']

This list can get very long. We can handle this by filtering out words that don't exist in the English language. To do this, we create a constant `WORDS` stores a list of all the words in an English dictionary (read in from a text file). The `utils` functions are included at the end of the notebook.

In [28]:
from utils import get_words
WORDS = get_words()

In [29]:
list(WORDS)[:8]

['cycled',
 'authoritative',
 'compilations',
 'propositioned',
 'sable',
 'bathers',
 'summation',
 'uglier']

We can now make the function `filter_unknown` which removes all words not in `WORDS`

In [30]:
def filter_unknown(words):
    return set(word for word in words if word in WORDS)

In [31]:
filter_unknown(one_edit_from("dog"))

{'bog',
 'cog',
 'dag',
 'dig',
 'do',
 'dob',
 'doc',
 'doe',
 'dog',
 'dogs',
 'doh',
 'don',
 'dos',
 'dot',
 'dug',
 'fog',
 'hog',
 'jog',
 'log',
 'tog',
 'wog'}

This is a far more manageable list, and is more useful as well because we don't want to correct misspelled words into words that don't exist in the English language.

To expand our search-space, we'll also include candidates that are two simple edits from the original word. This function, `two_edits_from` just stacks calls to `one_edit_from`

In [32]:
def two_edits_from(word):
    return set(
        second_edit
        for edit in one_edit_from(word)
        for second_edit in one_edit_from(edit)
    )

In [33]:
list(filter_unknown(two_edits_from("dog")))[:8]

['doh', 'loo', 'bogy', 'den', 'deb', 'sow', 'mob', 'lop']

In [34]:
filter_unknown(two_edits_from("example"))

{'ample', 'examine', 'example', 'examples', 'sample', 'trample'}

Tying it all together, we write the function `get_candidates` which accepts a word `w` and returns `w` if `w` is known, else all known words one edit from `w`. If there are no known words one edit from `w` then it returns all known words two edits from `w`. Failing that, it just returns `w`.

This function is meant to represent `P(w|c)` as it returns the candidates most likely to be the intended word. However, it falsely assumes that any candidate one edit away from the original word is infinitely more likely to be intended than a word two edits away. This isn't always true. For example, if `w` is "rember" then our model thinks "member" is infinitely more likely than "remember" which is clearly not true.

In [35]:
def get_candidates(word):
    if word in WORDS:
        return [word]
    
    return (
        filter_unknown(one_edit_from(word))
        or filter_unknown(two_edits_from(word))
        or [word]
    )

## Evaluating P(c)
To calculate the `p(c)` for any word `c`, we will use a large corpus of text and count the occurrences of `c` within the text and divide it by the total amount of words in the corpus. This will give us a good, if a bit crude, estimate of the probability of `c` showing up in our text. Our corpus will be a text file containing a large sample of ebooks from Project Gutenberg. We create a Counter that reads in all the words accumulates counts for each word.

In [36]:
from utils import get_word_counts
WORD_COUNTS = get_word_counts()

In [37]:
WORD_COUNTS["the"]

2272721

In [38]:
WORD_COUNTS["monopoly"]

141

In [39]:
WORD_COUNTS["horticulture"]

39

In [40]:
WORD_COUNTS.most_common(8)

[('the', 2272721),
 ('of', 1241239),
 ('and', 1239810),
 ('to', 1038194),
 ('a', 820395),
 ('in', 667929),
 ('i', 588076),
 ('that', 528540)]

We can now create the function `probability_of` which returns our estimate of `p(c)` given candidate word `c`

In [41]:
def probability_of(word):
    return WORD_COUNTS[word] / sum(WORD_COUNTS.values())

In [42]:
probability_of("the") # Large

0.05866879067049132

In [43]:
probability_of("monopoly") # Not large

3.639821819105502e-06

## Putting it together
We can now generate candidates and evaluate their probability of occurring. From these ingredients we can define a simple function to return the candidate with the maximum probability, completing our spelling corrector.

In [44]:
def correct(word):
    return max(get_candidates(word), key=probability_of)

In [45]:
misspellings = ["speling", "computinga", "clasrom", "ptyhon", "prgrammin", "jonahtan"]
for misspelling in misspellings:
    print(f"{misspelling} -> {correct(misspelling)}")

speling -> spelling
computinga -> computing
clasrom -> classroom
ptyhon -> python
prgrammin -> programming
jonahtan -> jonahtan


And it works! Though it apparently doesn't like names, can you figure out why? (Hint: Think about how we're filtering out unknown words)

I hope you found this notebook insightful into how spelling correction works at a base level. I've always found it interesting to see how technologies we use every day might work and put together this notebook to share that interest.

Please reach out if you have any questions or suggestions!

## Utility functions
These are the `utils` functions that we imported throughout the notebook. They generally just handle reading files.

In [46]:
import re
from collections import Counter
from os import listdir
from random import shuffle


def only_words(text: str):
    return re.findall(r"[^_\W]+", text.lower())


def get_words():
    with open("./datasets/dictionary.txt") as reader:
        return set(only_words(reader.read()))


def get_word_counts():
    with open("./datasets/corpus.txt") as reader:
        return Counter(only_words(reader.read()))


## Attributions
- This notebook was inspired by Peter Norvig's brilliant tutorial on the same subject: http://www.norvig.com/spell-correct.html
- The dictionary file was sourced from SCOWL's 12Dicts package: http://wordlist.aspell.net/12dicts/
- The Gutenberg corpus was compiled from Shibamouli Lahiri's work: https://web.eecs.umich.edu/~lahiri/gutenberg_dataset.html